In [1]:
from sklearn.preprocessing import MinMaxScaler

from torch.utils.data import TensorDataset
from torch.utils.data import ConcatDataset
from torch.utils.data import DataLoader

import torch.optim as optim
import torch.nn as nn
import torchvision
import torch

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 

/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <F0D48035-EF9E-3141-9F63-566920E60D7C> /Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <44B645FB-F027-3EE5-86D7-DBF8E2FC6264> /Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [2]:
tensor_model = torchvision.transforms.ToTensor()

trainset = torchvision.datasets.MNIST(root="./data", train=True,  transform=tensor_model, download=True)
testset  = torchvision.datasets.MNIST(root="./data", train=False, transform=tensor_model, download=True)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader  = DataLoader(testset,  batch_size=128, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
class BiLSMT(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, seq_length, num_classes, device):
        super(BiLSMT, self).__init__()
        self.device = device
        self.hidden_size = hidden_size
        self.num_layers  = num_layers
        self.seq_length  = seq_length
        self.lstm        = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc          = nn.Linear(seq_length * hidden_size * 2, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(self.device)

        # print(h0.shape, c0.shape, x.shape)

        out, _ = self.lstm(x, (h0, c0))
        out = out.reshape(-1, self.seq_length * self.hidden_size * 2)
        out = self.fc(out)
        return out

In [4]:
device = torch.device("mps")

sequence_length = trainset.data.size(1)
input_size      = trainset.data.size(2)
num_layers      = 2
hidden_size     = 12
num_classes     = 10

In [5]:
model = BiLSMT(input_size, hidden_size, num_layers, sequence_length, num_classes, device)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-3)

In [6]:
for epoch in range(51):
    correct = 0
    total = 0
    for data in trainloader:
        optimizer.zero_grad()
        inputs, labels = torch.squeeze(data[0]).to(device), data[1].to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.detach(), 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print("[%d] Train Acc : %.2f\tTrain Loss : %.2f" % (epoch, 100 * correct / total, loss / len(trainloader)))

[0] Train Acc : 47.53	Train Loss : nan
[1] Train Acc : 0.00	Train Loss : nan


# Evalutation

In [ ]:
def accuracy(dataloader):
    correct = 0
    total = 0
    with torch.no_grad():
        model.eval()
        for data in dataloader:
            inputs, labels = data[0].to(device).view(128, 28, 28), data[1].to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    model.train()
    return acc

In [ ]:
train_acc = accuracy(trainloader)
test_acc = accuracy(testloader)
print("Train Acc : %.2f\tTest Acc : %.2f" % (train_acc, test_acc))